In [1]:
import os
import gpxpy
import gpxpy.gpx
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import pandas as pd
import time

In [3]:
def split_gpx_into_segments_with_addresses(input_gpx_file, segment_length_km, output_directory, output_excel_file):
    # Créer le dossier de sortie s'il n'existe pas
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Charger le fichier GPX
    with open(input_gpx_file, "r") as f:
        gpx = gpxpy.parse(f)
    print(f"Fichier GPX chargé : {input_gpx_file}")
    
    # Initialiser les variables
    current_segment = gpxpy.gpx.GPXTrackSegment()
    segments = [current_segment]
    current_distance = 0.0
    segment_start_points = []  # Liste pour stocker les points de départ dans l'ordre
    is_new_segment = True  # Variable pour marquer le début d'un nouveau segment

    # Parcourir les points du fichier GPX
    for track in gpx.tracks:
        for segment in track.segments:
            for i in range(len(segment.points) - 1):
                point1 = segment.points[i]
                point2 = segment.points[i + 1]

                # Si c'est le début d'un nouveau segment, enregistrer son point de départ
                if is_new_segment:
                    segment_start_points.append((point1.latitude, point1.longitude))
                    is_new_segment = False  # Réinitialiser après avoir enregistré le point de départ

                # Calculer la distance entre deux points
                distance = geodesic((point1.latitude, point1.longitude), (point2.latitude, point2.longitude)).km
                current_distance += distance

                # Ajouter le point au segment actuel
                current_segment.points.append(point1)

                # Si la distance cumulée atteint le segment_length_km, créer un nouveau segment
                if current_distance >= segment_length_km:
                    current_segment.points.append(point2)  # Ajouter le dernier point du segment
                    current_segment = gpxpy.gpx.GPXTrackSegment()
                    segments.append(current_segment)
                    current_distance = 0.0
                    is_new_segment = True  # Marquer le début d'un nouveau segment

            # Ajouter le dernier point du segment
            current_segment.points.append(segment.points[-1])

    print(f"Nombre de segments créés : {len(segments)}")

    # Géocodage inverse pour obtenir les adresses
    geolocator = Nominatim(user_agent="gpx_segment_splitter", timeout=5)  # Augmenter le timeout

    # Fonction pour effectuer le géocodage inverse avec des tentatives de réessai
    def reverse_geocode_with_retry(geolocator, lat, lon, retries=3, delay=2):
        for attempt in range(retries):
            try:
                return geolocator.reverse((lat, lon))
            except Exception as e:
                if attempt < retries - 1:
                    time.sleep(delay)
                else:
                    raise e
    segment_addresses = []

    for idx, (lat, lon) in enumerate(segment_start_points):
        try:
            location = reverse_geocode_with_retry(geolocator, lat, lon)
            address = location.address if location else "Adresse introuvable"
        except Exception as e:
            address = f"Erreur : {e}"
        segment_addresses.append({
            "Segment": idx + 1,
            "Latitude": lat,
            "Longitude": lon,
            "Adresse": address
        })
        time.sleep(1)  # Pause de 1 seconde entre les requêtes

    print("Adresses géocodées pour chaque segment")
    # Créer un fichier Excel avec les adresses
    df = pd.DataFrame(segment_addresses)
    df.to_excel(output_excel_file, index=False)
    print(f"Fichier Excel enregistré sous : {output_excel_file}")


    # Créer un nouveau fichier GPX pour chaque segment
    for idx, segment in enumerate(segments):
        new_gpx = gpxpy.gpx.GPX()
        new_track = gpxpy.gpx.GPXTrack()
        new_gpx.tracks.append(new_track)
        new_track.segments.append(segment)

        output_file = os.path.join(output_directory, f"relai_{idx + 1}.gpx")
        with open(output_file, "w") as f:
            f.write(new_gpx.to_xml())
    print(f"Segments GPX enregistrés dans : {output_directory}")

In [4]:
# Exemple d'utilisation
input_gpx_file = "Circuit_complet.gpx"  # Fichier GPX modifié
segment_length_km = 10  # Longueur des segments en km
output_directory = "Relais_gpx"  # Dossier de sortie pour les fichiers GPX
output_excel_file = "segments_addresses.xlsx"  # Fichier Excel pour les adresses

split_gpx_into_segments_with_addresses(input_gpx_file, segment_length_km, output_directory, output_excel_file)

Fichier GPX chargé : Circuit_complet.gpx
Nombre de segments créés : 191
Adresses géocodées pour chaque segment
Fichier Excel enregistré sous : segments_addresses.xlsx
Segments GPX enregistrés dans : Relais_gpx
